In [2]:
#input C, K, D
fread = open("transport mat.txt", "r") #netdata.txt
S_D = fread.readline()
S_D = S_D.split(' ') #change to \t if tab spaced
sup = int(S_D[0])
dem = int(S_D[1].strip())
#print(E)
K = [0 for i in range(sup)]
D = [0 for i in range(dem)]
lines = fread.readlines()
C = [ [ 0 for i in range(dem) ] for j in range(sup) ]
s = 0
for line in lines:
    t_h = line.split(' ') #change to \t if tab spaced
    if (s == sup):
        break
    for i in range(dem):
        C[i][s] = float(t_h[i])
    K[s] = float(t_h[dem].replace("\n", ""))
    s = s + 1
for i in range(dem):
    D[i] = float(t_h[i])
print(K)
print(D)
print(C)
fread.close()

[10.0, 12.0]
[14.0, 13.0]
[[1.0, 2.0], [2.0, 3.0]]


In [2]:
import math
def sum_matrix(C):
    r = len(C)
    c = len(C[0])
    s = 0
    for i in range(r):
        for j in range(c):
            s = s + C[i][j]
    return s

# Northwest_corner
def northwest_corner(C, K, D):
    rmax = len(K)
    cmax = len(D)
    x = [[0 for i in range(cmax)] for j in range(rmax)]
    cost = 0
    while(sum_matrix(C) > 0):
        c = cmax
        for i in range(rmax):
            r = i
            flag = 0
            for j in range(cmax):
                if (C[i][j] > 0 and j < c):
                    c = j
                    flag = 1
            if (flag):
                break
        x[r][c] = min(K[r], D[c])
        cost = cost + x[r][c]*C[r][c]
        K[r] = K[r] - x[r][c]
        D[c] = D[c] - x[r][c]
        if (K[r] == 0):
            C[r] = [0 for i in range(cmax)]
        if (D[c] == 0):
            for i in range(rmax):
                C[i][c] = 0    
    return x, cost

In [20]:
#Min cost method
def min_cost(C, K, D):
    rmax = len(K)
    cmax = len(D)
    x = [[0 for i in range(cmax)] for j in range(rmax)]
    cost = 0
    #itr = 1
    while(sum(D) > 0):
        #print(itr)
        mc = math.inf
        for i in range(rmax):
            if (K[i] == 0):
                continue
            for j in range(cmax):
                if (D[j] == 0):
                    continue
                if (C[i][j] < mc):
                    mc = C[i][j]
                    r = i
                    c = j
        #print(r, c)
        x[r][c] = min(K[r], D[c])
        cost = cost + x[r][c]*C[r][c]
        #print(cost)
        K[r] = K[r] - x[r][c]
        D[c] = D[c] - x[r][c]
        if (K[r] == 0):
            for i in range(cmax):
                C[i][r] = 0
        if (D[c] == 0):
            for i in range(rmax):
                C[i][c] = 0    
        #print(C)
        #print(K)
        #print(D)
        #itr = itr + 1
        #if (itr > 10):
        #    break
    return x, cost

In [43]:
#Vogel
def first_min(row, arr):
    n = len(row)
    fmin = math.inf
    for i in range(n):
        if (row[i] < fmin and arr[i] > 0):
            fmin = row[i]
    return fmin
    
def sec_min(row, fmin, find, arr):
    n = len(row)
    smin = math.inf
    for i in range(n):
        if (row[i] < smin and row[i] >= fmin and i!= find and arr[i] > 0):
            smin = row[i]
    return smin    
    
def vogel(C, K, D):
    rmax = len(K)
    cmax = len(D)
    x = [[0 for i in range(cmax)] for j in range(rmax)]
    cost = 0
    suppliers_left = rmax
    demands_unmet = cmax
    while(suppliers_left > 1 and demands_unmet > 1):
        #print('here')
        max_penalty = -1
         #iteration row-wise
        for i in range(rmax):
            if (K[i] > 0):
                row = C[i]
                #print(row)
                c1min = first_min(row, D)
                m = row.index(c1min)
                c2min = sec_min(row, c1min, m, D)
                #print(c1min, c2min)
                row_penalty = c2min - c1min
                if (row_penalty > max_penalty):
                    max_penalty = row_penalty
                    frm = i
                    to = m
        #print(K)
        #print(max_penalty)
        
        #iteration col-wise
        #print(D)
        for j in range(cmax):
            if (D[j] > 0):
                col = [x[j] for x in C]
                #print(col)
                c1min = first_min(col, K)
                m = col.index(c1min)
                c2min = sec_min(col, c1min, m, K)
                #print(c1min, c2min)
                col_penalty = c2min - c1min
                if (col_penalty > max_penalty):
                    max_penalty = col_penalty
                    frm = m
                    to = j
        
        #print(D)
        #print(max_penalty)
        #print(frm, to)
        
        #update 1
        x[frm][to] = min(K[frm], D[to])
        cost = cost + C[frm][to]*x[frm][to]
        #print(cost)
        K[frm] = K[frm] - x[frm][to]
        D[to] = D[to] - x[frm][to]
        
        if (K[frm] == 0):
            for j in range(cmax):
                C[frm][j] = 0
            suppliers_left = suppliers_left - 1
        
        if (D[to] == 0):
            for i in range(rmax):
                C[i][to] = 0
            demands_unmet = demands_unmet - 1 
        
    #update2
    if (suppliers_left == 1):
        for i in range(rmax):
            if (K[i] > 0):
                frm = i
                #break
        for j in range(cmax):
            x[frm][j] = D[j]
            cost = cost + C[frm][j]*x[frm][j]
        #print('sl1')
        #print(cost)
    else:
        for j in range(cmax):
            if (D[j] > 0):
                to = j
                #break
        for i in range(rmax):
            x[i][to] = K[i]
            cost = cost + C[i][to]*x[i][to]
        #print('slnot1')
        #print(cost)
        
    
    return x, cost

In [49]:
C = [[5, 1, 8], [2, 4, 0], [3, 6, 7]]
K = [12, 14, 4]
D = [9, 10, 11]

print('Northwest corner')
x, cost = northwest_corner(C, K, D)
for i in range(len(K)):
    print(x[i])
#print(x)
print(cost)

Northwest corner
[9, 3, 0]
[0, 7, 0]
[0, 0, 4]
104


In [50]:
C = [[5, 1, 8], [2, 4, 0], [3, 6, 7]]
K = [12, 14, 4]
D = [9, 10, 11]

print('Min cost')
x, cost = min_cost(C, K, D)
for i in range(len(K)):
    print(x[i])
print(cost)

Min cost
[2, 10, 0]
[3, 0, 11]
[4, 0, 0]
38


In [51]:
C = [[5, 1, 8], [2, 4, 0], [3, 6, 7]]
K = [12, 14, 4]
D = [9, 10, 11]

print('Vogel')
x, cost = vogel(C, K, D)
for i in range(len(K)):
    print(x[i])
print(cost)

Vogel
[2, 10, 0]
[3, 0, 11]
[4, 0, 0]
38


In [65]:
#greedy outperformed by northwest
C = [[20, 30, 40], [5, 10, 15]]
K = [30, 10]
D = [15, 15, 10]

print('Northwest corner')
x, cost = northwest_corner(C, K, D)
print(x)
print(cost)

C = [[20, 30, 40], [5, 10, 15]]
K = [30, 10]
D = [15, 15, 10]

print('Min cost')
x, cost = min_cost(C, K, D)
print(x)
print(cost)

Northwest corner
[[15, 15, 0], [0, 0, 10]]
900
Min cost
[[5, 15, 10], [10, 0, 0]]
1000


In [67]:
#vogel outperformed by greedy
C = [[20, 35, 50], [15, 11, 10], [21, 36, 30]]
K = [18, 20, 2]
D = [11, 19, 10]

print('vogel')
x, cost = vogel(C, K, D)
print(x)
print(cost)

C = [[20, 35, 50], [15, 11, 10], [21, 36, 30]]
K = [18, 20, 2]
D = [11, 19, 10]

print('Min cost')
x, cost = min_cost(C, K, D)
print(x)
print(cost)

vogel
[[11, 0, 7], [0, 19, 1], [0, 0, 2]]
849
Min cost
[[11, 7, 0], [0, 10, 10], [0, 2, 0]]
747
